In [1]:
import pandas as pd
import datetime
from geopy.geocoders import Nominatim
import requests
import random
from time import sleep
from os import listdir, rename
from os.path import isfile, join
import tkinter as tk
from PIL import ImageTk, Image
import tensorflow.compat.v1 as tf
import numpy as np
from object_detection.utils import ops as utils_ops
import os
import person_detector
import webbrowser

In [2]:
# Create a wrapper to call the API easily by defining classes
TINDER_URL = "https://api.gotinder.com"
geolocator = Nominatim(user_agent="auto-tinder")
PROF_FILE = "./profiles.txt"

class Person(object):

    def __init__(self, data, api):
        self._api = api

        self.id = data["_id"]
        self.name = data.get("name", "Unknown")

        self.bio = data.get("bio", "")
        self.distance = data.get("distance_mi", 0) / 1.60934

        self.birth_date = datetime.datetime.strptime(data["birth_date"], '%Y-%m-%dT%H:%M:%S.%fZ') if data.get(
            "birth_date", False) else None
        self.gender = ["Male", "Female", "Unknown"][data.get("gender", 2)]

        self.images = list(map(lambda photo: photo["url"], data.get("photos", [])))

        self.jobs = list(
            map(lambda job: {"title": job.get("title", {}).get("name"), "company": job.get("company", {}).get("name")}, data.get("jobs", [])))
        self.schools = list(map(lambda school: school["name"], data.get("schools", [])))

        if data.get("pos", False):
            self.location = geolocator.reverse(f'{data["pos"]["lat"]}, {data["pos"]["lon"]}')


    def __repr__(self):
        return f"{self.id}  -  {self.name} ({self.birth_date.strftime('%d.%m.%Y')})"


    def like(self):
        return self._api.like(self.id)

    def dislike(self):
        return self._api.dislike(self.id)

    def download_images(self, folder=".", sleep_max_for=0):
        with open(PROF_FILE, "r") as f:
            lines = f.readlines()
            if self.id in lines:
                return
        with open(PROF_FILE, "a") as f:
            f.write(self.id+"\r\n")
        index = -1
        for image_url in self.images:
            index += 1
            req = requests.get(image_url, stream=True)
            if req.status_code == 200:
                with open(f"{folder}/{self.id}_{self.name}_{index}.jpeg", "wb") as f:
                    f.write(req.content)
            sleep(random.random()*sleep_max_for)
    
    def predict_likeliness(self, classifier, sess):
        ratings = []
        for image in self.images:
            req = requests.get(image, stream=True)
            tmp_filename = f"./images/tmp/run.jpg"
            if req.status_code == 200:
                with open(tmp_filename, "wb") as f:
                    f.write(req.content)
            img = person_detector.get_person(tmp_filename, sess)
            if img:
                img = img.convert('L')
                img.save(tmp_filename, "jpeg")
                certainty = classifier.classify(tmp_filename)
                pos = certainty["positive"]
                ratings.append(pos)
        ratings.sort(reverse=True)
        ratings = ratings[:5]
        if len(ratings) == 0:
            return 0.001
        return ratings[0]*0.6 + sum(ratings[1:])/len(ratings[1:])*0.4



class tinderAPI():

    def __init__(self, token):
        self._token = token

    def profile(self):
        data = requests.get(TINDER_URL + "/v2/profile?include=account%2Cuser", headers={"X-Auth-Token": self._token}).json()
        return Profile(data["data"], self)

    def matches(self, limit=10):
        data = requests.get(TINDER_URL + f"/v2/matches?count={limit}", headers={"X-Auth-Token": self._token}).json()
        return list(map(lambda match: Person(match["person"], self), data["data"]["matches"]))

    def like(self, user_id):
        data = requests.get(TINDER_URL + f"/like/{user_id}", headers={"X-Auth-Token": self._token}).json()
        return {
            "is_match": data["match"],
            "liked_remaining": data["likes_remaining"]
        }

    def dislike(self, user_id):
        requests.get(TINDER_URL + f"/pass/{user_id}", headers={"X-Auth-Token": self._token}).json()
        return True

    def nearby_persons(self):
        data = requests.get(TINDER_URL + "/v2/recs/core", headers={"X-Auth-Token": self._token}).json()
        return list(map(lambda user: Person(user["user"], self), data["data"]["results"]))

class Classifier():
    def __init__(self, graph, labels):

        self._graph = self.load_graph(graph)
        self._labels = self.load_labels(labels)

        self._input_operation = self._graph.get_operation_by_name("import/Placeholder")
        self._output_operation = self._graph.get_operation_by_name("import/final_result")

        self._session = tf.Session(graph=self._graph)

    def classify(self, file_name):
        t = self.read_tensor_from_image_file(file_name)

        # Open up a new tensorflow session and run it on the input
        results = self._session.run(self._output_operation.outputs[0], {self._input_operation.outputs[0]: t})
        results = np.squeeze(results)

        # Sort the output predictions by prediction accuracy
        top_k = results.argsort()[-5:][::-1]

        result = {}
        for i in top_k:
            result[self._labels[i]] = results[i]

        # Return sorted result tuples
        return result

    def close(self):
        self._session.close()


    @staticmethod
    def load_graph(model_file):
        graph = tf.Graph()
        graph_def = tf.GraphDef()
        with open(model_file, "rb") as f:
            graph_def.ParseFromString(f.read())
        with graph.as_default():
            tf.import_graph_def(graph_def)
        return graph

    @staticmethod
    def load_labels(label_file):
        label = []
        proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
        for l in proto_as_ascii_lines:
            label.append(l.rstrip())
        return label

    @staticmethod
    def read_tensor_from_image_file(file_name,
                                    input_height=299,
                                    input_width=299,
                                    input_mean=0,
                                    input_std=255):
        input_name = "file_reader"
        file_reader = tf.read_file(file_name, input_name)
        image_reader = tf.image.decode_jpeg(
            file_reader, channels=3, name="jpeg_reader")
        float_caster = tf.cast(image_reader, tf.float32)
        dims_expander = tf.expand_dims(float_caster, 0)
        resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
        normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
        sess = tf.Session()
        result = sess.run(normalized)
        return result

In [10]:
# Get and download pictures for our dataset

token = ''

# Instantiate the API
api = tinderAPI(token)

# While true, loop through all people and download pics
while True:
    persons = api.nearby_persons()
    for person in persons:
        print(person)
        person.download_images(folder='./Images/Unclassified', sleep_max_for=random.random()*3)
        sleep(random.random()*10)

    sleep(random.random()*10)


nessa (01.11.1994)
5c4e6854dc46b61100b362d3  -  Cassandra (01.11.1998)
5f1bed2187b3a00100a8ff6d  -  Taylor (01.11.1998)
5f8284632396b40100e7a4be  -  Katrina (01.11.1996)
5edc86c454760a0100af512b  -  Danielle (01.11.1995)
5f9a695c1e40e60100ead5b4  -  Kali (01.11.1995)
5f7bc63127615e0100bc217f  -  Lauren (01.11.1995)
59d31bb1eb6999bd70acfa3a  -  Connie (01.11.1994)
5e22bdbaec574601007e6c2b  -  Robin (01.11.1996)
5f8f68796827e101007841ee  -  Ann (01.11.1995)
5f8a4e88607aa4010090d75e  -  Genesis (01.11.1994)
5cbb70309107ca1500fa9e1a  -  Liana (01.11.1996)
5e7c3e14dfb4390100c8e6ab  -  Annah (01.11.1994)
5dd84d824330f801002d221b  -  Kristine (01.11.1996)
56b56a41adbd6f4f76495615  -  Jocelyn (01.11.1996)
5f1bed2187b3a00100a8ff6d  -  Taylor (01.11.1998)
5d4653b52d44671600d665f9  -  Tara (01.11.1996)
5f606616067a040100646551  -  Jocelyn (01.11.1995)
5f6ae59c7c19430100fd8528  -  Abbie (01.11.1996)
536b02bfec24cc156900f5f1  -  Tara (01.11.1995)
5e7e721cc82cc601001a71b3  -  Jackie (01.11.1995)
59a

ChunkedEncodingError: ("Connection broken: ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)", ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [3]:
# Image classifier GUI

IMAGE_FOLDER = "./images/unclassified"

images = [f for f in listdir(IMAGE_FOLDER) if isfile(join(IMAGE_FOLDER, f))]
unclassified_images = filter(lambda image: not (image.startswith("0_") or image.startswith("1_")), images)
current = None

def next_img():
    global current, unclassified_images
    try:
        current = next(unclassified_images)
    except StopIteration:
        root.quit()
    #print(current)
    pil_img = Image.open(IMAGE_FOLDER+"/"+current)
    width, height = pil_img.size
    max_height = 1000
    if height > max_height:
        resize_factor = max_height / height
        pil_img = pil_img.resize((int(width*resize_factor), int(height*resize_factor)), resample=Image.LANCZOS)
    img_tk = ImageTk.PhotoImage(pil_img)
    img_label.img = img_tk
    img_label.config(image=img_label.img)

def positive(arg):
    global current
    rename(IMAGE_FOLDER+"/"+current, IMAGE_FOLDER+"/1_"+current)
    next_img()

def negative(arg):
    global current
    rename(IMAGE_FOLDER + "/" + current, IMAGE_FOLDER + "/0_" + current)
    next_img()



root = tk.Tk()

img_label = tk.Label(root)
img_label.pack()
img_label.bind("<Button-1>", positive)
img_label.bind("<Button-3>", negative)

btn = tk.Button(root, text='Next image', command=next_img)

next_img() # load first image

root.mainloop()

TypeError: can only concatenate str (not "NoneType") to str

In [5]:
# Process and sort our images after they have been classified

IMAGE_FOLDER = "./images/unclassified"
POS_FOLDER = "./images/classified/positive"
NEG_FOLDER = "./images/classified/negative"



detection_graph = person_detector.open_graph()

images = [f for f in os.listdir(IMAGE_FOLDER) if os.path.isfile(os.path.join(IMAGE_FOLDER, f))]
positive_images = filter(lambda image: (image.startswith("1_")), images)
negative_images = filter(lambda image: (image.startswith("0_")), images)

with detection_graph.as_default():
    with tf.Session() as sess:

        for pos in positive_images:

            try:
                old_filename = IMAGE_FOLDER + "/" + pos
                new_filename = POS_FOLDER + "/" + pos[:-5] + ".jpg"
                if not os.path.isfile(new_filename):
                    img = person_detector.get_person(old_filename, sess)
                    if not img:
                        continue
                    img = img.convert('L')
                    img.save(new_filename, "jpeg")

            except:
                next

        for neg in negative_images:

            try:
                old_filename = IMAGE_FOLDER + "/" + neg
                new_filename = NEG_FOLDER + "/" + neg[:-5] + ".jpg"
                if not os.path.isfile(new_filename):
                    img = person_detector.get_person(old_filename, sess)
                    if not img:
                        continue
                    img = img.convert('L')
                    img.save(new_filename, "jpeg")

            except:
                next


In [3]:
# Run on autopilot

token = ''

# Instantiate the API
api = tinderAPI(token)

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

detection_graph = person_detector.open_graph()
with detection_graph.as_default():
    with tf.Session() as sess:

        classifier = Classifier(graph="./tf/training_output/retrained_graph.pb",
                                labels="./tf/training_output/retrained_labels.txt")

        sent_likes = 0

        max_likes = 20

        while sent_likes < max_likes:
            #try:
            persons = api.nearby_persons()

            for person in persons:

                try:
                    #print(person)

                    print('Likes Sent: ' + str(sent_likes))

                    webbrowser.open(person.images[0])

                    score = person.predict_likeliness(classifier, sess)

                    #print("ID: ", person.id)
                    print("Name: ", person.name)
                    #print("Schools: ", person.schools)
                    #print("Images: ", person.images)
                    print(score)

                    if score > 0.84:
                        res = person.like()
                        print("LIKE")

                        webbrowser.open(person.images[0])

                        sent_likes += 1

                        
                    else:
                        res = person.dislike()
                        print("DISLIKE")

                    print('-----------------------------')
                except Exception:
                    pass

classifier.close()

Likes Sent: 0
Name:  Noelle
0.8280117988586426
DISLIKE
-----------------------------
Likes Sent: 0


KeyboardInterrupt: 